In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')
import random
from tensorflow.keras.models import load_model

# Load trained model and other necessary files
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess user input
def clean_up_sentence(sentence):
    # Tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to create bag of words from user input
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    # Initialize bag of words
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict class of user input and generate response
def predict_class(sentence):
    # Filter out predictions below a certain threshold
    p = bag_of_words(sentence)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # Sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(ints, intents_json):
    if ints:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for intent in list_of_intents:
            if intent['tag'] == tag:
                return random.choice(intent['responses'])
    else:
        return "I'm sorry, I'm not sure how to respond to that."


# Function to chat with the chatbot
def chatbot_response(sentence):
    ints = predict_class(sentence)
    res = get_response(ints, intents)
    return res

# Chat with the chatbot
print("Chatbot: Hello! I’m ViewPoint, a bot specializing in POV productions. How may I assist you today? (type 'quit' to end)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Chatbot: Hello! I’m ViewPoint, a bot specializing in POV productions. How may I assist you today? (type 'quit' to end)
You: what is pov
1/1 [==============================] - 0s 135ms/step
Chatbot: Welcome to POV (Point of View), a dynamic startup committed to revolutionizing digital landscapes through innovative content and marketing strategies. Founded by Mr. Prakhar Dixit, Ms. Diksha Singh, and Mr. Swapnil Gupta, POV has quickly emerged as a powerhouse in the realm of social media management, performance marketing, and public relations. Our tagline is “Collaboration is the new Sexy”.
You: what does pov serve
1/1 [==============================] - 0s 21ms/step
Chatbot: I'm sorry, I'm not sure how to respond to that.
You: what do pov serve
1/1 [==============================] - 0s 23ms/step
Chatbot: POV crafts strategic content journeys and amplifies unique brand narratives to fuel organic growth in the digital realm.
You: what is pov achevements
1/1 [==============================] -

KeyboardInterrupt: Interrupted by user